In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,precision_recall_curve, \
roc_auc_score,roc_curve,recall_score,classification_report, f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.svm import OneClassSVM
from icecream import ic
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
df = pd.read_csv('train.csv')
df['credit_line_utilization'] = df['credit_line_utilization'].str.replace(',', '.').astype(float)
columns = df.iloc[:, 1:-1].columns
columns

Index(['age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more'],
      dtype='object')

In [3]:
X, y = df[columns], df.iloc[:, -1]
Counter(y)

Counter({0: 67148, 1: 5013})

In [12]:
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X, y)

In [13]:
Counter(y_rus)

Counter({0: 5013, 1: 5013})

In [18]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe9 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier())
    ])
    pipe9.fit(X_train, y_train)
    y_predicted=pipe9.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

0.7672805782856598
0.7434676463805456


In [19]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe9.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

0.8799602365224706


In [17]:
?RandomForestClassifier

In [20]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe10 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe10.fit(X_train, y_train)
    y_predicted=pipe10.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

0.7658024588273651
0.7513443233055801


In [22]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe9.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

0.88276978593381


In [26]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
# from sklearn.feature_selection import SelectKBest

In [33]:
pipe12 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('rf', RandomForestClassifier())
    ])
params = {
    'rf__n_estimators': [120, 140],
    'rf__max_depth': [30, 50]
}

RF_gs = GridSearchCV(pipe12, param_grid=params, scoring='roc_auc', cv=3)
RF_gs.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
RF_gs.score(X_test, y_test)